In [2]:
'''
Author: Zhen Liu lzhen.dev@outlook.com
CreateDate: Do not edit
LastEditors: Zhen Liu lzhen.dev@outlook.com
LastEditTime: 2024-03-02
Description: 

Copyright (c) 2024 by HernandoR lzhen.dev@outlook.com, All Rights Reserved. 
'''
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score
import random
import numpy as np
import pandas as pd
import wandb

from pytorch_tabular.utils import make_mixed_dataset, print_metrics, load_covertype_dataset
from pytorch_tabular.utils import get_balanced_sampler, get_class_weighted_cross_entropy

import os
%load_ext autoreload
%autoreload 2

In [3]:
wandb.login()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: hernando (lzhen-ntu). Use `wandb login --relogin` to force relogin


True

In [4]:
# data, cat_col_names, num_col_names = make_mixed_dataset(
#     task="classification", 
#     n_samples=10000, 
#     n_features=8, 
#     n_categories=4, 
#     weights=[0.8], 
#     random_state=42)
# target_col='target'


In [5]:

Feature_Explain=pd.read_excel('data/Amex_ori/Amex Campus.xlsx')
Feature_target_cols = Feature_Explain['Feature Name'][Feature_Explain['Extended description']=='Target'].to_list()
Feature_cat_col_names = Feature_Explain['Feature Name'][Feature_Explain['Feature Type']=='categorical'].to_list()
Feature_num_col_names = Feature_Explain['Feature Name'][Feature_Explain['Variable Type'] =='numeric'].to_list()

In [6]:
data = pd.read_parquet('data/train.parquet')

cols=data.columns.to_list()
target_cols = [col for col in cols if col in Feature_target_cols]
if len(target_cols) == 0:
    print('No target column found')
    exit()
elif len(target_cols) > 1:
    target_col=target_cols[0]
cat_col_names = [col for col in cols if col in Feature_cat_col_names]
num_col_names = [col for col in cols if col in Feature_num_col_names]

# target_col=cols.unite(Feature_target_cols)

# target_col = 'activation'

In [7]:

data.head()
print(f"target_col={target_col}, cat_col_names={cat_col_names}, num_col_names={num_col_names}")

target_col=activation, cat_col_names=['merchant_profile_01'], num_col_names=['customer_digital_activity_02', 'customer_profile_01', 'customer_profile_02', 'customer_profile_03', 'customer_profile_04', 'customer_spend_06', 'customer_spend_07', 'distance_05', 'ind_recommended']


In [8]:


train, test = train_test_split(data, random_state=42)
train, val = train_test_split(train, random_state=42)

In [11]:
# data['customer_digital_activity_01'].value_counts(normalize=True)
# data.describe()
# len(train)

customer_digital_activity_01
1.000000    1.791388e-01
0.000000    8.756835e-02
0.500000    1.442840e-02
0.333333    9.474261e-03
0.666667    9.028062e-03
                ...     
0.425287    8.176630e-07
0.563218    5.723641e-07
0.839080    4.088315e-07
0.060241    2.452989e-07
0.969388    8.176630e-08
Name: proportion, Length: 2418, dtype: float64

# Importing the Library

In [12]:
from pytorch_tabular import TabularModel
from pytorch_tabular.models import CategoryEmbeddingModelConfig
from pytorch_tabular.config import DataConfig, OptimizerConfig, TrainerConfig, ExperimentConfig
from pytorch_tabular.models.common.heads import LinearHeadConfig

In [13]:
results = []

## Define the Configs


In [14]:
data_config = DataConfig(
    target=[target_col], #target should always be a list. Multi-targets are only supported for regression. Multi-Task Classification is not implemented
    continuous_cols=num_col_names,
    categorical_cols=cat_col_names,
    num_workers=10,
)
trainer_config = TrainerConfig(
    auto_lr_find=True, # Runs the LRFinder to automatically derive a learning rate
    batch_size=5210,
    max_epochs=100,
    early_stopping="valid_loss", # Monitor valid_loss for early stopping
    early_stopping_mode = "min", # Set the mode as min because for val_loss, lower is better
    early_stopping_patience=5, # No. of epochs of degradation training will wait before terminating
    checkpoints="valid_loss", # Save best checkpoint monitoring val_loss
    checkpoints_path = 'checkpoints/', # Save the checkpoint in the experiment directory
    load_best=True, # After training, load the best checkpoint
    accelerator="mps",
    trainer_kwargs={
        "enable_progress_bar":True, 
        # "progress_bar_refresh_rate":1
    }
)
optimizer_config = OptimizerConfig()

head_config = LinearHeadConfig(
    layers="", # No additional layer in head, just a mapping layer to output_dim
    dropout=0.1,
    initialization="kaiming"
).__dict__ # Convert to dict to pass to the model config (OmegaConf doesn't accept objects)

EXP_PROJECT_NAME = "pytorch-tabular-covertype"


In [15]:

model_config = CategoryEmbeddingModelConfig(
    task="classification",
    layers="1024-512-512",  # Number of nodes in each layer
    activation="LeakyReLU", # Activation between each layers
    head = "LinearHead", #Linear Head
    head_config = head_config, # Linear Head Config
    learning_rate = 1e-3,
    metrics=["f1_score","accuracy"], 
    metrics_params=[{"num_classes":2},{}], # f1_score needs num_classes
    metrics_prob_input=[True, False] # f1_score needs probability scores, while accuracy doesn't
)

# experiment_config = ExperimentConfig(
#     project_name=EXP_PROJECT_NAME,
#     run_name="CategoryEmbeddingModel",
#     exp_watch="gradients",
#     log_target="wandb",
#     log_logits=True
# )



## Training the Model 

In [16]:
tabular_model = TabularModel(
    data_config=data_config,
    model_config=model_config,
    optimizer_config=optimizer_config,
    trainer_config=trainer_config,
    # experiment_config=experiment_config,
    verbose=False,
    # suppress_lightning_logger=True,

)

In [17]:
sampler = get_balanced_sampler(train[target_col].values.ravel())

tabular_model.fit(train=train, validation=val, train_sampler=sampler)

Seed set to 42
GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/opt/homebrew/Caskroom/miniforge/base/envs/AMEX/lib/python3.11/site-packages/pytorch_lightning/callbacks/model_checkpoint.py:639: Checkpoint directory /Users/lz/Codes/AE2024/checkpoints exists and is not empty.
/opt/homebrew/Caskroom/miniforge/base/envs/AMEX/lib/python3.11/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:436: Consider setting `persistent_workers=True` in 'train_dataloader' to speed up the dataloader worker initialization.
/opt/homebrew/Caskroom/miniforge/base/envs/AMEX/lib/python3.11/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:436: Consider setting `persistent_workers=True` in 'val_dataloader' to speed up the dataloader worker initialization.


Finding best initial lr:   0%|          | 0/100 [00:00<?, ?it/s]

Trainer was signaled to stop but the required `min_epochs=1` or `min_steps=None` has not been met. Training will continue...
`Trainer.fit` stopped: `max_steps=100` reached.
Learning rate set to 0.0001
Restoring states from the checkpoint path at /Users/lz/Codes/AE2024/.lr_find_ac0065aa-0319-467b-8674-73537f4e0638.ckpt
Restored all states from the checkpoint at /Users/lz/Codes/AE2024/.lr_find_ac0065aa-0319-467b-8674-73537f4e0638.ckpt


┏━━━┳━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━┓
┃   ┃ Name             ┃ Type                      ┃ Params ┃
┡━━━╇━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━┩
│ 0 │ _backbone        │ CategoryEmbeddingBackbone │  834 K │
│ 1 │ _embedding_layer │ Embedding1dLayer          │  2.6 K │
│ 2 │ head             │ LinearHead                │  1.0 K │
│ 3 │ loss             │ CrossEntropyLoss          │      0 │
└───┴──────────────────┴───────────────────────────┴────────┘

Trainable params: 838 K                                                                                            
Non-trainable params: 0                                                                                            
Total params: 838 K                                                                                                
Total estimated model params size (MB): 3

Output()

In [ ]:
result = tabular_model.evaluate(test)

In [ ]:
result = {k: float(v) for k,v in result[0].items()}
result["mode"] = "Normal"

results.append(result)

## Custom Sampler

PyTorch Tabular also allows custom batching strategy through Custom Samplers  which comes in handy when working with imbalanced data.

Although you can use any sampler, Pytorch Tabular has a few handy utility functions which takes in the target array and implements WeightedRandomSampler using inverse frequency sampling to combat imbalance. This is analogous to preprocessing techniques like Under or OverSampling in traditional ML systems.

In [ ]:
tabular_model = TabularModel(
    data_config=data_config,
    model_config=model_config,
    optimizer_config=optimizer_config,
    trainer_config=trainer_config,
    verbose=False
)
sampler = get_balanced_sampler(train[target_col].values.ravel())

tabular_model.fit(train=train, validation=val, train_sampler=sampler)


In [ ]:
result = tabular_model.evaluate(test)

In [ ]:
result = {k: float(v) for k,v in result[0].items()}
result["mode"] = "Balanced Sampler"

results.append(result)

## Custom Weighted Loss

If Samplers were like Over/Under Sampling, Custom Weighted Loss is similar to `class_weights`. Depending on the problem, one of these might help you with imbalance. You can easily make calculate the class_weights and provide them to the CrossEntropyLoss using the parameter `weight`. To make this easier, PyTorch Tabular has a handy utility method which calculates smoothed class weights and initializes a weighted loss. Once you have that loss, it's just a matter of passing it to the 1fit1 method using the `loss` parameter.

In [ ]:
tabular_model = TabularModel(
    data_config=data_config,
    model_config=model_config,
    optimizer_config=optimizer_config,
    trainer_config=trainer_config,
    verbose=False
)
weighted_loss = get_class_weighted_cross_entropy(train["target"].values.ravel(), mu=0.1)

tabular_model.fit(train=train, validation=val, loss=weighted_loss)


In [ ]:
result = tabular_model.evaluate(test)

In [ ]:
result = {k: float(v) for k,v in result[0].items()}
result["mode"] = "Class Weights"

results.append(result)

In [ ]:
res_df = pd.DataFrame(results).T
res_df.columns = res_df.iloc[-1]
res_df = res_df.iloc[:-1].astype(float)
res_df.style.highlight_min(color="lightgreen",axis=1)